In [1]:
import math
import time
import numpy as np
import pandas as pd
from tqdm import trange

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
# from torch.utils.tensorboard import SummaryWriter

# K折交叉验证
def k_fold(data_X, data_y, name, device='cpu',
           k=5, num_epochs=50, learning_rate=1e-3, weight_decay=0.0, batch_size=128):
    train_l_sum, valid_l_sum = [], []
    train_total_time = 0.0
    for i in range(k):
        data = get_k_fold_data(k, i, data_X, data_y) # X_train, y_train, X_valid, y_valid
        net = get_net(device)
        train_ls, valid_ls, train_time = train(name, i, net, *data, num_epochs, learning_rate, weight_decay, batch_size, device)
        index = valid_ls.index(max(valid_ls)) # 返回列表最大值的索引
        # train_l_sum += train_ls[index]
        # valid_l_sum += valid_ls[index]
        train_l_sum.append(train_ls[index])
        valid_l_sum.append(valid_ls[index])
        train_total_time += train_time
        print(f'折{i + 1}, train_time(second): {train_time}' )
        print(f'折{i + 1}, 训练log accuracy{float(train_ls[index]):f}, '
              f'验证log accuracy{float(valid_ls[index]):f}')

    return np.mean(train_l_sum), np.mean(valid_l_sum), np.std(valid_l_sum), train_total_time / k

def get_k_fold_data(k, i, X, y):
    assert k > 1
    fold_size = X.shape[0] // k
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid


def build_dataloader(data_x, data_y, BATCH_SIZE=128, is_train=True):
    # 构造一个Pytorch数据迭代器
    data_tensor = TensorDataset(data_x, data_y)                    
    data_loader = DataLoader(dataset=data_tensor, batch_size=BATCH_SIZE, shuffle=is_train)
    return data_loader

In [2]:
# 训练+验证
def train(name, i, net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size, device):
    # writer = SummaryWriter(log_dir=f'logs/{name}/折{i+1}')
    # writer = {'train_loss': SummaryWriter(log_dir=f'logs/{name}/折{i+1}/train'),
    #           'test_loss': SummaryWriter(log_dir=f'logs/{name}/折{i+1}/test')}
    train_ls, test_ls = [], []
    train_iter = build_dataloader(train_features, train_labels, batch_size) # train_loader
    test_iter = build_dataloader(test_features, test_labels, batch_size) # test_loader
    # Adam优化算法
    optimizer = torch.optim.Adam(net.parameters(),
                                 lr = learning_rate,
                                 weight_decay = weight_decay)
    criterion = nn.CrossEntropyLoss(reduction='mean')
    total_time = 0.0
    # max_correct = 0.0
    BEST_LOSS = np.inf # 学习率衰减
    patience_counter = 0    
    for epoch in trange(1, num_epochs+1):
        net.train()
        start = time.time()
        total_loss = 0.0
        for ii, batch in enumerate(train_iter):
            X, y = batch
            X, y = X.unsqueeze(1).to(device=device), y.to(device=device)
            pre = net(X)
            loss = criterion(pre, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss +=loss.item()
        
        end = time.time()
        time_elapsed = end - start
        # writer.add_scalar("Time/train", time_elapsed, epoch)
        total_time += time_elapsed

        net.eval()
        for NAME, loader in [("train", train_iter), ("test", test_iter)]:
            loss_sum = 0.0
            correct = 0.0
            total = 0.0
            for x_input, y_label in loader:
                x_input, y_label = x_input.unsqueeze(1).to(device=device), y_label.to(device=device)
                y_pred = net(x_input)
                LOSS = criterion(y_pred, y_label)
                loss_sum += LOSS.item()
                _, predicted = torch.max(y_pred, dim=1)

                total += y_label.shape[0]
                correct += int((predicted == y_label).sum())
                
            if NAME == 'train':
                train_ls.append(correct/total)
                # writer['train_loss'].add_scalar("Loss", loss_sum/len(loader), epoch)
                
            else:
                # 保存权重
                # if correct > max_correct:
                #     max_correct = correct
                #     torch.save(net, f'{name}.pth')
                    
                test_ls.append(correct/total)
                # writer['test_loss'].add_scalar("Loss", loss_sum/len(loader), epoch)
                
                # 学习率衰减
                if loss_sum < BEST_LOSS:
                    BEST_LOSS = loss_sum
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= 5 :
                        learning_rate = 0.2*learning_rate 
                        # print(learning_rate)
                        optimizer = torch.optim.Adam(net.parameters(),
                                                    lr = learning_rate,
                                                    weight_decay = weight_decay)
                        patience_counter = 0 
            # writer.add_scalar(name, correct/total, epoch)
            
    # writer['train_loss'].close()
    # writer['test_loss'].close()   
    return train_ls, test_ls, total_time

In [3]:
from modules.data_processing import *
# from modules.setup_seed import *
# from modules.cross_validation import k_fold

device = (torch.device('cuda') if torch.cuda.is_available()
            else torch.device('cpu'))
print(f"Training on device {device}.")
data_x = np.load('./data/Bearing_Zhaomh/SNR-N5/data_x.npy')
data_y = np.load('./data/Bearing_Zhaomh/SNR-N5/data_y.npy')
data_x, data_y = data_processing(data_x, data_y)

Training on device cuda.


In [4]:
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

# __all__ = ('CBAM', 'SENet', 'autopad', 'Conv', 'Bottleneck, 'C2f', 'ARNet')

# CBAM 注意力
class ChannelAttention(nn.Module):
    """Channel-attention module https://github.com/open-mmlab/mmdetection/tree/v3.0.0rc1/configs/rtmdet."""
    def __init__(self, channels: int) -> None:
        super().__init__()
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Conv1d(channels, channels, 1, 1, 0, bias=True)
        self.act = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * self.act(self.fc(self.pool(x)))

class SpatialAttention(nn.Module):
    """Spatial-attention module."""
    def __init__(self, kernel_size=7):
        """Initialize Spatial-attention module with kernel size argument."""
        super().__init__()
        assert kernel_size in (3, 7), 'kernel size must be 3 or 7'
        padding = 3 if kernel_size == 7 else 1
        self.cv1 = nn.Conv1d(2, 1, kernel_size, padding=padding, bias=False)
        self.act = nn.Sigmoid()

    def forward(self, x):
        """Apply channel and spatial attention on input for feature recalibration."""
        return x * self.act(self.cv1(torch.cat([torch.mean(x, 1, keepdim=True), torch.max(x, 1, keepdim=True)[0]], 1))) 

class CBAM(nn.Module):
    """Convolutional Block Attention Module."""
    def __init__(self, c1, kernel_size=7):  # ch_in, kernels
        super().__init__()
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        """Applies the forward pass through C1 module."""
        return self.spatial_attention(self.channel_attention(x))
# CBAM 注意力


# SE 注意力
class SENet(nn.Module):
    def __init__(self, channels, ratio=16):
        super(SENet, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels//ratio, False),
            nn.ReLU(),
            nn.Linear(channels//ratio, channels, False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        N, c, L = x.size()
        avg = self.avg_pool(x).view([N, c])
        fc = self.fc(avg).view([N, c, 1])

        return x*fc
# SE 注意力


def autopad(k, p=None, d=1):  # kernel, padding, dilation
    """Pad to 'same' shape outputs."""
    if d > 1:
        k = d * (k - 1) + 1 if isinstance(k, int) else [d * (x - 1) + 1 for x in k]  # actual kernel-size
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
    return p


class Conv(nn.Module):
    """Standard convolution with args(ch_in, ch_out, kernel, stride, padding, groups, dilation, activation)."""
    default_act = nn.SiLU() # default activation

    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, d=1, act=True):
        """Initialize Conv layer with given arguments including activation."""
        super().__init__()
        self.conv = nn.Conv1d(c1, c2, k, s, autopad(k, p, d), groups=g, dilation=d, bias=False)
        self.bn = nn.BatchNorm1d(c2)
        self.act = self.default_act if act is True else act if isinstance(act, nn.Module) else nn.Identity()

    def forward(self, x):
        """Apply convolution, batch normalization and activation to input tensor."""
        return self.act(self.bn(self.conv(x)))
    
    
class Bottleneck(nn.Module):  
    """DarknetBottleneck"""
    def __init__(self, c1, c2, shortcut=True, e=0.5,
                 k1=19, d1=2, k2=17, d2=4):
        super().__init__()
        c_ = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, c_, k=k1, p=autopad(k=k1, d=d1), d=d1)
        self.cv2 = Conv(c_, c2, k=k2, p=autopad(k=k2, d=d2), d=d2)
        self.add = shortcut and c1 == c2

    def forward(self, x):
        return x + self.cv2(self.cv1(x)) if self.add else self.cv2(self.cv1(x))
    

class C2f(nn.Module):  # CSPLayer_2Conv
    """Faster Implementation of CSP Bottleneck with 2 convolutions."""
    def __init__(self, c1, c2, n=1, shortcut=True, e=0.5,
                 k1=19, d1=2, k2=17, d2=4):
        super().__init__()
        self.c = int(c2 * e)  # hidden channels
        self.cv1 = Conv(c1, 2 * self.c, 1, 1)
        self.cv2 = Conv((2 + n) * self.c, c2, 1)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, e, k1, d1, k2, d2) for _ in range(n))

    def forward(self, x):
        """Forward pass through C2f layer."""
        y = list(self.cv1(x).chunk(2, 1))   
        y.extend(m(y[-1]) for m in self.m)  
        return self.cv2(torch.cat(y, 1))  # Concat -> ConvModule
    
class ARNet(nn.Module): # Attention_ReshuffleNet
    def __init__(self, channels, ratio=1):
        super(ARNet, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels//ratio, False),
            nn.SiLU(),
            nn.Linear(channels//ratio, channels, False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        B, C, L = x.size()
        avg = self.avg_pool(x).view([B, C])
        fc = self.fc(avg).view([B, C, 1])
        x = x*fc
        indices = torch.argsort(fc, dim=1, descending=False).expand(B, C, L)
        x = torch.gather(x, dim=1, index=indices)
        return x
    
class dilated_c2f1(nn.Module):
    def __init__(self, c1, c2, n=1, shortcut=True, e=1,
                 k1=19, d1=2, k2=17, d2=4):
        super().__init__()
        self.c = c1//2
        self.ar = ARNet(channels=c1)        
        self.cv = Conv((2 + n) * self.c, c2, 1)
        self.m = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, e, k1, d1, k2, d2) for _ in range(n))

    def forward(self, x):
        y = list(self.ar(x).chunk(2, 1))
        y.extend(m(y[-1]) for m in self.m)  
        return self.cv(torch.cat(y, 1))
    
    
class dilated_c2f2(nn.Module):
    def __init__(self, c1, c2, n=1, shortcut=True, e=1,
                 k1=19, d1=2, k2=17, d2=4):
        super().__init__()
        self.c = c1//3
        self.ar1 = ARNet(channels=c1)
        self.ar2 = ARNet(channels=(c1-self.c))         
        self.cv = Conv((5 + n) * self.c, c2, 1)
        self.m1 = nn.ModuleList(Bottleneck(2*self.c, 2*self.c, shortcut, e, k1, d1, k2, d2) for _ in range(1))
        self.m2 = nn.ModuleList(Bottleneck(self.c, self.c, shortcut, e, k1, d1, k2, d2) for _ in range(n))

    def forward(self, x):
        y = list(torch.tensor_split(self.ar1(x), (self.c,), dim=1))
        y.extend(m1(y[-1]) for m1 in self.m1)
        x = self.ar2(y[-1])
        y.pop()
        y.extend(x.chunk(2, 1))
        y.extend(m2(y[-1]) for m2 in self.m2)
        return self.cv(torch.cat(y, 1))

In [5]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    c = 48
    self.conv0 = Conv(1, c, k=1, s=1)

    self.conv1 = Conv(c, 2*c, k=3, s=2, p=1)
    self.c2f1 = dilated_c2f1(2*c, 2*c, n=1, shortcut=True, e=1,
                             k1=39, d1=2, k2=35, d2=5)
    
    self.conv2 = Conv(2*c, 3*c, k=3, s=2, p=1)
    self.c2f2 = dilated_c2f2(3*c, 3*c, n=1, shortcut=True, e=1,
                             k1=29, d1=2, k2=27, d2=5)
    
    self.pool = nn.AdaptiveAvgPool1d(1)
    self.drop = nn.Dropout(p=0.)
    self.fc = nn.Linear(3*c, 5)
  
  def forward(self, x):
    # x.shape: (batch, 1024, 1)
    x = self.conv0(x)
    
    x = self.conv1(x)
    x = self.c2f1(x)
    
    x = self.conv2(x)
    x = self.c2f2(x)
    
    x = self.pool(x).flatten(1)
    x = self.drop(x)
    x = self.fc(x) 
    return x

def get_net(device):
    net = Model().to(device=device)
    return net

train_a, valid_a, valid_s, average_time = k_fold(data_x, data_y, 'Model', device,
                        k=5, num_epochs=70, learning_rate=0.001)
# print('train_a', train_a)
print('valid_a', valid_a)
print('valid_s', valid_s)
# print('time', average_time)    
# 升序   

100%|██████████| 70/70 [07:11<00:00,  6.16s/it]


折1, train_time(second): 301.071964263916
折1, 训练log accuracy0.999375, 验证log accuracy0.940000


100%|██████████| 70/70 [07:10<00:00,  6.15s/it]


折2, train_time(second): 304.9759786128998
折2, 训练log accuracy1.000000, 验证log accuracy0.940000


100%|██████████| 70/70 [07:12<00:00,  6.17s/it]


折3, train_time(second): 303.623033285141
折3, 训练log accuracy0.996875, 验证log accuracy0.940000


100%|██████████| 70/70 [07:16<00:00,  6.23s/it]


折4, train_time(second): 305.4885232448578
折4, 训练log accuracy0.996875, 验证log accuracy0.921250


100%|██████████| 70/70 [07:05<00:00,  6.08s/it]

折5, train_time(second): 295.38654804229736
折5, 训练log accuracy0.999062, 验证log accuracy0.941250
valid_a 0.9365
valid_s 0.007640353394968046


In [6]:
class ARNet(nn.Module): # Attention_ReshuffleNet
    def __init__(self, channels, ratio=1):
        super(ARNet, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels//ratio, False),
            nn.SiLU(),
            nn.Linear(channels//ratio, channels, False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        B, C, L = x.size()
        avg = self.avg_pool(x).view([B, C])
        fc = self.fc(avg).view([B, C, 1])
        x = x*fc
        indices = torch.argsort(fc, dim=1, descending=True).expand(B, C, L)
        x = torch.gather(x, dim=1, index=indices)
        return x
    
train_a, valid_a, valid_s, average_time = k_fold(data_x, data_y, 'Model', device,
                        k=5, num_epochs=70, learning_rate=0.001)
# print('train_a', train_a)
print('valid_a', valid_a)
print('valid_s', valid_s)
# print('time', average_time)  
# 降序

100%|██████████| 70/70 [07:10<00:00,  6.15s/it]


折1, train_time(second): 296.68719005584717
折1, 训练log accuracy0.995313, 验证log accuracy0.928750


100%|██████████| 70/70 [07:05<00:00,  6.08s/it]


折2, train_time(second): 292.10922932624817
折2, 训练log accuracy0.975000, 验证log accuracy0.930000


100%|██████████| 70/70 [07:16<00:00,  6.24s/it]


折3, train_time(second): 300.46922159194946
折3, 训练log accuracy0.897188, 验证log accuracy0.897500


100%|██████████| 70/70 [07:24<00:00,  6.35s/it]


折4, train_time(second): 301.6461844444275
折4, 训练log accuracy0.986563, 验证log accuracy0.921250


100%|██████████| 70/70 [07:07<00:00,  6.11s/it]

折5, train_time(second): 292.6357831954956
折5, 训练log accuracy0.920312, 验证log accuracy0.912500
valid_a 0.9179999999999999
valid_s 0.012005207203542988


In [7]:
class ARNet(nn.Module): # Attention_ReshuffleNet
    def __init__(self, channels, ratio=1):
        super(ARNet, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels//ratio, False),
            nn.SiLU(),
            nn.Linear(channels//ratio, channels, False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        B, C, L = x.size()
        avg = self.avg_pool(x).view([B, C])
        fc = self.fc(avg).view([B, C, 1])
        x = x*fc
        # indices = torch.argsort(fc, dim=1, descending=True).expand(B, C, L)
        # x = torch.gather(x, dim=1, index=indices)
        return x
    
train_a, valid_a, valid_s, average_time = k_fold(data_x, data_y, 'Model', device,
                        k=5, num_epochs=70, learning_rate=0.001)
# print('train_a', train_a)
print('valid_a', valid_a)
print('valid_s', valid_s)
# print('time', average_time)  
# 无排序

100%|██████████| 70/70 [07:08<00:00,  6.12s/it]


折1, train_time(second): 300.936137676239
折1, 训练log accuracy1.000000, 验证log accuracy0.922500


100%|██████████| 70/70 [06:45<00:00,  5.79s/it]


折2, train_time(second): 285.88668489456177
折2, 训练log accuracy1.000000, 验证log accuracy0.933750


100%|██████████| 70/70 [06:51<00:00,  5.88s/it]


折3, train_time(second): 289.5573847293854
折3, 训练log accuracy1.000000, 验证log accuracy0.925000


100%|██████████| 70/70 [06:56<00:00,  5.95s/it]


折4, train_time(second): 293.63865399360657
折4, 训练log accuracy1.000000, 验证log accuracy0.932500


100%|██████████| 70/70 [06:56<00:00,  5.94s/it]

折5, train_time(second): 294.43532633781433
折5, 训练log accuracy1.000000, 验证log accuracy0.945000
valid_a 0.9317500000000001
valid_s 0.007889866919029728


In [8]:
class ARNet(nn.Module): # Attention_ReshuffleNet
    def __init__(self, channels, ratio=1):
        super(ARNet, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels//ratio, False),
            nn.SiLU(),
            nn.Linear(channels//ratio, channels, False),
            nn.Sigmoid(),
        )

    def forward(self, x):
        B, C, L = x.size()
        avg = self.avg_pool(x).view([B, C])
        fc = self.fc(avg).view([B, C, 1])
        # x = x*fc
        indices = torch.argsort(fc, dim=1, descending=True).expand(B, C, L)
        x = torch.gather(x, dim=1, index=indices)
        return x
    
train_a, valid_a, valid_s, average_time = k_fold(data_x, data_y, 'Model', device,
                        k=5, num_epochs=70, learning_rate=0.001)
# print('train_a', train_a)
print('valid_a', valid_a)
print('valid_s', valid_s)
# print('time', average_time)  
# 无强化

100%|██████████| 70/70 [06:43<00:00,  5.76s/it]


折1, train_time(second): 280.00688123703003
折1, 训练log accuracy0.994062, 验证log accuracy0.918750


100%|██████████| 70/70 [06:45<00:00,  5.79s/it]


折2, train_time(second): 283.43673157691956
折2, 训练log accuracy0.997812, 验证log accuracy0.927500


100%|██████████| 70/70 [06:57<00:00,  5.97s/it]


折3, train_time(second): 292.08569717407227
折3, 训练log accuracy0.996563, 验证log accuracy0.923750


100%|██████████| 70/70 [06:41<00:00,  5.74s/it]


折4, train_time(second): 279.4677209854126
折4, 训练log accuracy0.987812, 验证log accuracy0.930000


100%|██████████| 70/70 [06:42<00:00,  5.75s/it]

折5, train_time(second): 274.9848575592041
折5, 训练log accuracy0.965625, 验证log accuracy0.937500
valid_a 0.9275
valid_s 0.006274950199005588


In [9]:
class ARNet(nn.Module): # Attention_ReshuffleNet
    def __init__(self, channels, ratio=1):
        super(ARNet, self).__init__()
        # self.avg_pool = nn.AdaptiveAvgPool1d(1)
        # self.fc = nn.Sequential(
        #     nn.Linear(channels, channels//ratio, False),
        #     nn.SiLU(),
        #     nn.Linear(channels//ratio, channels, False),
        #     nn.Sigmoid(),
        # )

    def forward(self, x):
        # B, C, L = x.size()
        # avg = self.avg_pool(x).view([B, C])
        # fc = self.fc(avg).view([B, C, 1])
        # x = x*fc
        # indices = torch.argsort(fc, dim=1, descending=True).expand(B, C, L)
        # x = torch.gather(x, dim=1, index=indices)
        return x
    
train_a, valid_a, valid_s, average_time = k_fold(data_x, data_y, 'Model', device,
                        k=5, num_epochs=70, learning_rate=0.001)
# print('train_a', train_a)
print('valid_a', valid_a)
print('valid_s', valid_s)
# print('time', average_time)  
# 无注意

100%|██████████| 70/70 [06:26<00:00,  5.53s/it]


折1, train_time(second): 273.7907338142395
折1, 训练log accuracy1.000000, 验证log accuracy0.916250


100%|██████████| 70/70 [06:31<00:00,  5.59s/it]


折2, train_time(second): 275.6815013885498
折2, 训练log accuracy1.000000, 验证log accuracy0.925000


100%|██████████| 70/70 [06:31<00:00,  5.59s/it]


折3, train_time(second): 273.21705961227417
折3, 训练log accuracy1.000000, 验证log accuracy0.922500


100%|██████████| 70/70 [06:28<00:00,  5.55s/it]


折4, train_time(second): 274.1193013191223
折4, 训练log accuracy1.000000, 验证log accuracy0.928750


100%|██████████| 70/70 [06:54<00:00,  5.93s/it]

折5, train_time(second): 288.4284772872925
折5, 训练log accuracy1.000000, 验证log accuracy0.936250
valid_a 0.9257500000000001
valid_s 0.006642665127793214
